# Download Email Events from Hubspot API

This app will download all email events from Hubspot API. It uses Campaign Statistics to decide what timeframe to begin downloading from.

Resources:
* MSSQL - Run DDL in the project against your database
* Schema named "hs" in database

Put the event types you want to download in the hs.DownloadableEventType table

In [1]:
import requests, json, time
import pandas as pd
import sqlalchemy
import pyodbc as odbc

import globvars
import functions
import campaigns
import events
import contacts

runningcallcount = 0

In [ ]:
# SQLAlchemy Connection
meta_engine = sqlalchemy.create_engine(globvars.dsnalchemy)

# Test the connection
connection = meta_engine.connect()
result = connection.execute("SELECT @@ServerName ServerName, DB_NAME() AS DatabaseName, CURRENT_USER AS LoggedInAs")
for row in result:
    print (row['ServerName'], row['DatabaseName'], row['LoggedInAs'])

In [ ]:
# PYODBC Connection
conn = odbc.connect(globvars.dsnpyodbc, autocommit=True)
curs = conn.cursor()
curs.execute("SELECT @@ServerName AS ServerName, DB_NAME() AS DatabaseName, CURRENT_USER AS LoggedInAs")
print curs.fetchone()

## Step 1: Setup the Download Run
I like to keep track of each run and when I did it

In [ ]:
curs.execute("INSERT INTO hs.DownloadRun(RunDate) VALUES(GETDATE())")
ident = curs.execute("SELECT SCOPE_IDENTITY()").fetchone()[0]

## Step 2: Get Campaign Stats

In [ ]:
# Get All the campaign statistics
numCalls = campaigns.get_all_campaignstats(noisy=True)

runningcallcount = runningcallcount + numCalls

## Step 3: Loop through the events we want to capture and download

In [5]:
events.get_campaign_allevents(113, 46169860, 1)

appId:113	campaignId:46169860	open:0	expected:0	calls:1
appId:113	campaignId:46169860	click:0	expected:0	calls:1
appId:113	campaignId:46169860	spamreport:0	expected:0	calls:1
appId:113	campaignId:46169860	bounce:0	expected:-11	calls:1
appId:113	campaignId:46169860	deferred:0	expected:0	calls:1
appId:113	campaignId:46169860	delivered:0	expected:-2	calls:1
appId:113	campaignId:46169860	dropped:0	expected:-1499	calls:1
appId:113	campaignId:46169860	forward:0	expected:0	calls:1
appId:113	campaignId:46169860	print:0	expected:-1	calls:1
appId:113	campaignId:46169860	processed:0	expected:0	calls:1
appId:113	campaignId:46169860	sent:0	expected:-28	calls:1
appId:113	campaignId:46169860	statuschange:0	expected:0	calls:1


## Step 4: Store Stats History
Finalize the data

In [ ]:
df = dictToDF(campaignStats)
df['RunID'] = ident

df.to_sql(name="CampaignStatisticsHistory"
            , schema="hs"
            , con=meta_engine
            , index=False
            , if_exists="append"
            , dtype=globvars.COLUMNSCAMPAIGNSTATISTICSHISTORY)